<h1> Coursera_Capstone_Project</h1>

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

import seaborn as sb
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
print('imported')

In [ ]:
print("Hello Capstone Project Course!")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#Importing Dataset
data_df = pd.read_csv('data//data-collisions.csv', encoding='latin-1', low_memory=False)
data_df_shape_ori = data_df.shape
# print(data_df.loc[0,:])
# data_df.head(10)
print(data_df.isnull().sum(axis=0))

In [ ]:
data_df['SEVERITYDESC'].unique()

In [ ]:
#Check for and remove any rows with SEVERITYCODE = "0"
#Before removal get the total count of SEVERITYCODE
print("Give a run-down of the values present in the table for SEVERITYCODE:")
print(data_df["SEVERITYCODE"].value_counts())

In [ ]:
#Check for and remove any rows with SEVERITYDESC = "Unknown"
#Before removal get the total count of SEVERITYDESC
print("Give a run-down of the values present in the table for SEVERITYDESC:")
print(data_df["SEVERITYDESC"].value_counts())

In [ ]:
#drop unwanted columns

# col_set = ["SEVERITYCODE", "SEVERITYDESC", "SPEEDING","ROADCOND", "WEATHER", "ROADCOND", "LIGHTCOND", "INATTENTIONIND"]
# data_df = data_df[col_set]

drop_cols = ['OBJECTID', 'COLDETKEY', 'REPORTNO', 'STATUS', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'INCDATE', 'SDOTCOLNUM', 'ST_COLCODE']

data_df_01 = data_df.drop(columns=drop_cols)

data_df_01.head()

In [ ]:
#Find missing WEATHERCOND
# print(data_df_01["WEATHER"] == 'Unknown')
todrop1 = data_df_01["WEATHER"] == 'Unknown'
noweatherinfo = todrop1.values.sum()
print(noweatherinfo)

In [ ]:
#Find missing ROADCOND
todrop2 = data_df_01["ROADCOND"] == "Unknown"
noroadcondinfo = todrop2.values.sum()
print(noroadcondinfo)

In [ ]:
#Find missing LIGHTCOND
todrop3 = data_df_01["LIGHTCOND"] == "Unknown"
nolightinfo = todrop3.values.sum()
print(nolightinfo)

In [ ]:
#Collate these and remove
data_df_01["TODROP"] = 0
count_noweather = 0
count_noinfo = 0
for i in range(0,len(todrop1)):
    if todrop1[i] == True or todrop2[i] == True or todrop3[i] == True:
        data_df_01["TODROP"][i] = 1

In [ ]:
print(data_df_01["TODROP"].unique())

In [ ]:
print("There are "+str(noweatherinfo)+" accidents with no weather information.")
print("There are "+str(noroadcondinfo)+" accidents with no road condition information.")
print("There are "+str(nolightinfo)+" accidents with no information about light conditions.")
print("There are "+str(data_df_01["TODROP"].values.sum())+" accidents without one or more of the above.\n Deleting now...")

In [ ]:
#Delete the temporary column "TODROP" and re-index the data
shape0 = data_df_01.shape
todrop = data_df_01["TODROP"] == 1
data_df_01.drop(data_df_01.index[todrop], inplace=True)

In [ ]:
#Print info about DF shape before and after
print("Shape of DF before: "+str(shape0[0])+" x "+str(shape0[1]))
print("Shape of DF after: "+str(data_df_01.shape[0])+" x "+str(data_df_01.shape[1]))
print("The original DF had shape: "+str(data_df_shape_ori[0])+" x "+str(data_df_shape_ori[1]))

#Re-index dataframe to account for missing rows
data_df_01.reset_index(inplace=True)

In [ ]:
#Delete the temporary "TODROP" column from the DataFrame
if 'TODROP' in data_df_01:
    del data_df_01["TODROP"]
    del data_df_01["index"]

In [ ]:
#Both SEVERITYCODE & SEVERITYDESC count should match
print(data_df_01["SEVERITYCODE"].value_counts())
print(data_df_01["SEVERITYDESC"].value_counts())

In [ ]:
#Know the Speeding existing values and set them to numeric 0 & 1
data_df_01["SPEEDING"].unique()

In [ ]:
#set speeding values [nan, 'Y'] to numeric 0 & 1
data_df_01["SPEEDING"].replace(np.nan, 0, inplace=True)
data_df_01["SPEEDING"].replace("Y", 1, inplace=True)
data_df_01["SPEEDING"].unique()

In [ ]:
#Know the INATTENTIONIND existing values and set them to numeric 0 & 1
data_df_01["INATTENTIONIND"].unique()

In [ ]:
#set INATTENTIONIND values [nan, 'Y'] to numeric 0 & 1
data_df_01["INATTENTIONIND"].replace(np.nan, 0, inplace=True)
data_df_01["INATTENTIONIND"].replace("Y", 1, inplace=True)
data_df_01["INATTENTIONIND"].unique()

In [ ]:
#Know the UNDERINFL existing values and set them to numeric 0 & 1
data_df_01["UNDERINFL"].unique()

In [ ]:
# Set UNDERINFL values ['N', '0', nan, '1', 'Y'] to numeric 0, 1, 1
data_df_01["UNDERINFL"].replace(np.nan, 0, inplace=True)
data_df_01["UNDERINFL"].replace('N', 0, inplace=True)
data_df_01["UNDERINFL"].replace('0', 0, inplace=True)
data_df_01["UNDERINFL"].replace('1', 1, inplace=True)
data_df_01["UNDERINFL"].replace("Y", 1, inplace=True)
data_df_01["UNDERINFL"].unique()

In [ ]:
#PEDROWNOTGRNT & HITPARKEDCAR
data_df_01["PEDROWNOTGRNT"].replace(np.nan, 0, inplace=True)
data_df_01["PEDROWNOTGRNT"].replace("Y", 1, inplace=True)

data_df_01["HITPARKEDCAR"].replace("N", 0, inplace=True)
data_df_01["HITPARKEDCAR"].replace(np.nan, 0, inplace=True)
data_df_01["HITPARKEDCAR"].replace("Y", 1, inplace=True)

In [ ]:
# Know more about LIGHTCOND
data_df_01['LIGHTCOND'].describe()
data_df_01['LIGHTCOND'].value_counts()

In [ ]:
# Know more about WEATHER
data_df_01['WEATHER'].describe()
data_df_01['WEATHER'].value_counts()

In [ ]:
# Know more about ROADCOND
data_df_01['ROADCOND'].describe()
data_df_01['ROADCOND'].value_counts()

In [ ]:
# Know more about ADDRTYPE
data_df_01['ADDRTYPE'].describe()
data_df_01['ADDRTYPE'].value_counts()

In [ ]:
#Get dummies for one-hot encoding
weatherdummies = pd.get_dummies(data_df_01["WEATHER"])
lightconddummies = pd.get_dummies(data_df_01["LIGHTCOND"])
roadconddummies = pd.get_dummies(data_df_01["ROADCOND"])
addrtypedummies = pd.get_dummies(data_df_01["ADDRTYPE"])
# weatherdummies.head()
lightconddummies.head()
# roadconddummies.head()

In [ ]:
#There is an "other" column in each of the above DFs -- rename these to avoid confusion
#when the DataFrames are merged
weatherdummies.rename(columns={"Other": "OtherWeather"}, inplace=True)
lightconddummies.rename(columns={"Other": "OtherLight"}, inplace=True)
roadconddummies.rename(columns={"Other": "OtherRoadcond"}, inplace=True)
# weatherdummies.head()
lightconddummies.head()
# roadconddummies.head()

In [ ]:
#Lets have a look at the current state of the df
data_df_01.head()

In [ ]:
#Now use one-hot encoding on SDOT_COLCODE
colcodedummies = pd.get_dummies(data_df_01["SDOT_COLCODE"])
columns = colcodedummies.columns
for col in columns:
    colcodedummies = colcodedummies.rename(columns={col: 'SDOT_COLCODE_'+str(col)})

In [ ]:
#Lets have a look at the current state of the df
data_df_01.head()

In [ ]:
#Join columns to existing DataFrame
#data_frames = [df, weatherdummies, lightconddummies, roadconddummies, addrtypedummies]
data_df_01 = data_df_01.join(weatherdummies)
data_df_01 = data_df_01.join(lightconddummies)
data_df_01 = data_df_01.join(roadconddummies)
data_df_01 = data_df_01.join(addrtypedummies)
data_df_01 = data_df_01.join(colcodedummies)

In [ ]:
print(colcodedummies.shape)
colcodedummies.head(5)

In [ ]:

#Separate Date/Time from INCDTTM colum and create columns for Hour/Minute, Day/Month/Year
data_df_01["INC_DATE_TIME"] = pd.to_datetime(data_df_01["INCDTTM"])
data_df_01["MONTH"] = pd.DatetimeIndex(data_df_01["INC_DATE_TIME"]).month
data_df_01["DAY_OF_WEEK"] = data_df_01["INC_DATE_TIME"].dt.dayofweek
data_df_01["DAY_OF_MONTH"] = pd.DatetimeIndex(data_df_01["INC_DATE_TIME"]).day
data_df_01["YEAR"] = pd.DatetimeIndex(data_df_01["INC_DATE_TIME"]).year


In [ ]:
data_df_01.head(5)

In [ ]:
import time
from datetime import datetime, timedelta
import concurrent.futures
import multiprocessing
from tqdm.auto import tqdm, trange
num_processes = multiprocessing.cpu_count()

print("There are "+str(num_processes)+" logical cores on this system.")
global_starttime = time.time()

#Create new columns
data_df_01["TIME_ROUNDED_TO_NEAREST_HOUR"] = 0
data_df_01["DAY_STRING"] = 0
data_df_01["MONTH_STRING"] = 0
data_df_01["WEEKEND"] = 0

In [ ]:
data_df_01.head()

In [ ]:
print(data_df_01.shape[0])

In [ ]:
#Define some useful functions
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

def get_dow(day_int):
    if day_int == 0:
        dow = 'Mon'
    if day_int == 1:
        dow = 'Tue'
    if day_int == 2:
        dow = 'Wed'
    if day_int == 3:
        dow = 'Thu'
    if day_int == 4:
        dow = 'Fri'
    if day_int == 5:
        dow = 'Sat'
    if day_int == 6:
        dow = 'Sun'
    return dow

def get_month(month_int):
    if month_int == 1:
        monthstring = 'Jan'
    if month_int == 2:
        monthstring = 'Feb'
    if month_int == 3:
        monthstring = 'Mar'
    if month_int == 4:
        monthstring = 'Apr'
    if month_int == 5:
        monthstring = 'May'
    if month_int == 6:
        monthstring = 'Jun'
    if month_int == 7:
        monthstring = 'Jul'
    if month_int == 8:
        monthstring = 'Aug'
    if month_int == 9:
        monthstring = 'Sep'
    if month_int == 10:
        monthstring = 'Oct'
    if month_int == 11:
        monthstring = 'Nov'
    if month_int == 12:
        monthstring = 'Dec'
    return monthstring
     
def is_weekend(day_int):
    if day_int <= 4:
        isweekend = 0
    else:
        isweekend = 1
    return isweekend

start_time = time.time()
#Get the day of the week from its integer value (0-6)
print("Get the day of the week from its integer value (0-6):")
def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as pool:
        data_df_01["DAY_STRING"] = list(tqdm(pool.map(get_dow, data_df_01["DAY_OF_WEEK"], chunksize=10), total=data_df_01.shape[0]))

    #Get the name of the month from its integer value (1-12)
    print("Get the name of the month from its integer value (1-11):")
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as pool:
        data_df_01["MONTH_STRING"] = list(tqdm(pool.map(get_month, data_df_01["MONTH"], chunksize=10), total=data_df_01.shape[0]))
    
    #Round time to the nearest whole hour
    print("Round time to the nearest whole hour:")
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as pool:
        data_df_01["TIME_ROUNDED_TO_NEAREST_HOUR"] = list(tqdm(pool.map(hour_rounder, data_df_01["INC_DATE_TIME"], chunksize=10), total=data_df_01.shape[0]))
    
    #Check if an accident occurred on the weekend or not
    print("Check if an accident occurred on the weekend or not:")
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as pool:
        data_df_01["WEEKEND"] = list(tqdm(pool.map(is_weekend, data_df_01["DAY_OF_WEEK"], chunksize=10), total=data_df_01.shape[0]))

    elapsed_time = time.time() - start_time
    print("Elapsed time for parallel FOR loop (seconds): "+str(elapsed_time))

    #Now store the nearest hour to the time of the accident as an integer and delete the rounded Datetime
    data_df_01["HOUR_NEAREST"] = pd.DatetimeIndex(data_df_01["TIME_ROUNDED_TO_NEAREST_HOUR"]).hour
    
    del data_df_01["TIME_ROUNDED_TO_NEAREST_HOUR"]

if __name__ == '__main__':
    main()

In [ ]:
data_df_01.head(5)

In [ ]:
#Check the dataframe for any NaNs
print(data_df_01.isnull().sum(axis=0))

In [ ]:
##Missing geographical data
#Find null values and replace these with NaN
#Find NaNs and set these to spaces.
print("There are "+str(data_df_01["X"].isnull().sum())+" null location values. Will try to get X, Y data for these based on the locations of other accidents that share the same INTKEY.")

#Go through the accidents with no X,Y data and see if we can match them to others
#that share the same INTKEY
data_df_01["NOT_HAS_COORDS"] = data_df_01["X"].isnull()
count = 0
for i in range(0,len(data_df_01["X"])):
    if data_df_01["NOT_HAS_COORDS"][i] == True:
        if data_df_01["INTKEY"][i] > 0:
            #Find other accidents which share the same INTKEY
            matched_intersections = data_df_01.loc[data_df_01["INTKEY"] == data_df_01["INTKEY"][i]]
            #Do any of these have the required X,Y data?
            with_geodata = matched_intersections["X"].count()
            if with_geodata > 0:
                matched_intersections.head()
                count += 1

if "NOT_HAS_COORDS" in data_df_01:
    del data_df_01["NOT_HAS_COORDS"]
    
print("Added locations for "+str(count)+" accidents based on shared INTKEYS.")

In [ ]:
data_df_01.head(5)

In [43]:
#Keep the full dataframe, but create a copy which drops columns which will not be used for modelling
data = data_df_01.copy()
if 'INCKEY' in data_df_01:
    del data["INCKEY"]
    del data["INTKEY"]
    del data["ADDRTYPE"]
    del data["LOCATION"]
    del data["SEVERITYDESC"]
    del data["COLLISIONTYPE"]
    del data["INCDTTM"]
    del data["JUNCTIONTYPE"]
    del data["SDOT_COLDESC"]

In [44]:
data.head(5)

,ï»¿SEVERITYCODE,X,Y,SEVERITYCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,OtherWeather,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,OtherLight,Dry,Ice,Oil,OtherRoadcond,Sand/Mud/Dirt,Snow/Slush,Standing Water,Wet,Alley,Block,Intersection,SDOT_COLCODE_0,SDOT_COLCODE_11,SDOT_COLCODE_12,SDOT_COLCODE_13,SDOT_COLCODE_14,SDOT_COLCODE_15,SDOT_COLCODE_16,SDOT_COLCODE_18,SDOT_COLCODE_21,SDOT_COLCODE_22,SDOT_COLCODE_23,SDOT_COLCODE_24,SDOT_COLCODE_25,SDOT_COLCODE_26,SDOT_COLCODE_27,SDOT_COLCODE_28,SDOT_COLCODE_29,SDOT_COLCODE_31,SDOT_COLCODE_32,SDOT_COLCODE_33,SDOT_COLCODE_34,SDOT_COLCODE_35,SDOT_COLCODE_36,SDOT_COLCODE_44,SDOT_COLCODE_46,SDOT_COLCODE_47,SDOT_COLCODE_48,SDOT_COLCODE_51,SDOT_COLCODE_52,SDOT_COLCODE_53,SDOT_COLCODE_54,SDOT_COLCODE_55,SDOT_COLCODE_56,SDOT_COLCODE_58,SDOT_COLCODE_61,SDOT_COLCODE_64,SDOT_COLCODE_66,SDOT_COLCODE_68,SDOT_COLCODE_69,INC_DATE_TIME,MONTH,DAY_OF_WEEK,DAY_OF_MONTH,YEAR,TIME_ROUNDED_TO_NEAREST_HOUR,DAY_STRING,MONTH_STRING,WEEKEND
0,2,-122.323148,47.703140,2,2,0,0,2,11,0,0,Overcast,Wet,Daylight,0,0,Entering at angle,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013-03-27 14:54:00,3,2,27,2013,2013-03-27 15:00:00,Wed,Mar,0
1,1,-122.347294,47.647172,1,2,0,0,2,16,0,0,Raining,Wet,Dark - Street Lights On,0,0,From same direction - both going straight - bo...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2006-12-20 18:55:00,12,2,20,2006,2006-12-20 19:00:00,Wed,Dec,0
2,1,-122.334540,47.607871,1,4,0,0,3,14,0,0,Overcast,Dry,Daylight,0,0,One parked--one moving,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2004-11-18 10:20:00,11,3,18,2004,2004-11-18 10:00:00,Thu,Nov,0
3,1,-122.334803,47.604803,1,3,0,0,3,11,0,0,Clear,Dry,Daylight,0,0,From same direction - all others,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013-03-29 09:26:00,3,4,29,2013,2013-03-29 09:00:00,Fri,Mar,0
4,2,-122.306426,47.545739,2,2,0,0,2,11,0,0,Raining,Wet,Daylight,0,0,Entering at angle,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2004-01-28 08:04:00,1,2,28,2004,2004-01-28 08:00:00,Wed,Jan,0


In [45]:
print("The shape of the dataframe is:")
print(data.shape)

The shape of the dataframe is:
(175771, 97)


In [46]:
print("The data types are:")
data.dtypes

The data types are:


ï»¿SEVERITYCODE                          int64
X                                      float64
Y                                      float64
SEVERITYCODE                             int64
PERSONCOUNT                              int64
PEDCOUNT                                 int64
PEDCYLCOUNT                              int64
VEHCOUNT                                 int64
SDOT_COLCODE                             int64
INATTENTIONIND                           int64
UNDERINFL                                int64
WEATHER                                 object
ROADCOND                                object
LIGHTCOND                               object
PEDROWNOTGRNT                            int64
SPEEDING                                 int64
ST_COLDESC                              object
SEGLANEKEY                               int64
CROSSWALKKEY                             int64
HITPARKEDCAR                             int64
Blowing Sand/Dirt                        uint8
Clear        

In [47]:
print("Here is a count of how many null/NaN values there are in each column:")
print(data.isnull().sum(axis=0))

Here is a count of how many null/NaN values there are in each column:
ï»¿SEVERITYCODE                    0
X                               3964
Y                               3964
SEVERITYCODE                       0
PERSONCOUNT                        0
PEDCOUNT                           0
PEDCYLCOUNT                        0
VEHCOUNT                           0
SDOT_COLCODE                       0
INATTENTIONIND                     0
UNDERINFL                          0
WEATHER                         5067
ROADCOND                        4995
LIGHTCOND                       5121
PEDROWNOTGRNT                      0
SPEEDING                           0
ST_COLDESC                      4900
SEGLANEKEY                         0
CROSSWALKKEY                       0
HITPARKEDCAR                       0
Blowing Sand/Dirt                  0
Clear                              0
Fog/Smog/Smoke                     0
OtherWeather                       0
Overcast                           0
Partl

In [62]:
import folium

#Get the median latitude and logitude and centre the map there
lon_med = data["X"].mean()
lat_med = data["Y"].mean()

print(lat_med)
print(lon_med)

seattle_map = folium.Map(location=[lat_med, lon_med], zoom_start=11)
seattle_map



47.618959790793774
-122.33036557514542


In [66]:
import pyscreenshot as ImageGrab
from selenium import webdriver
im=ImageGrab.grab()
im.save('map1_seattle.png', format='png')
im=ImageGrab.grab(bbox=(500,250,1480,830))

In [67]:
#Read the Seattle geojson data
import json
import requests

#GeoJSON file obtained from https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/neighborhoods.geojson
#!wget https://raw.githubusercontent.com/seattleio/seattle-boundaries-data/master/data/neighborhoods.geojson
geofile = 'data//neighborhoods.geojson'

with open(geofile) as json_file:
    geodata = json.load(json_file)

print("There are "+str(len(geodata['features']))+" districts in the Seattle area:")

There are 175 districts in the Seattle area:


In [69]:
!pip install shapely

     |████████████████████████████████| 1.0 MB 658 kB/s eta 0:00:01


In [ ]:
#Convert latitude/longitude to point objects and find out which area they lie in
from shapely.geometry import shape, Point, Polygon
from shapely.ops import nearest_points
from time import strftime, gmtime
import random

    
def find_county(lon, lat):
    point = Point(lon, lat)
    found_nhood = 0
    '''Most accidents can be trivially located within a neighbourhood
       using the polygon.contains() function from shapely
    '''
    for i in range(0,len(geodata['features'])):
        polygon = shape(geodata['features'][i]['geometry'])
    if polygon.contains(point):
            nhood = geodata['features'][i]['properties']['name']
            found_nhood = 1
    if found_nhood == 0:
        '''Some accidents can't be located within neighbourhoods because
           they occur on major highways which mark neighbourhood boundaries:
           the "no man's land" problem. Let's deal with these by assigning
           them to the _nearest_ neighbourhood
        '''
        nhoods = []
        distances = np.zeros(len(geodata['features']))
        for i in range(0,len(geodata['features'])):
            polygon = shape(geodata['features'][i]['geometry'])
            distances[i] = point.distance(polygon)
            nhoods.append(geodata['features'][i]['properties']['name'])
            
        min_distance = np.where(distances == np.amin(distances)) 
        try:
            #Usually this finds the nearest district
            min_distance_index = int(min_distance[0])
        except:
            '''But sometimes the accident is exactly equidistant between
               two neighbourhood boundaries. In this case, choose one
               of the districts at random to avoid biasing the sample
            '''
            min_distance_index = int(random.choice(min_distance[0]))
        nhood = nhoods[min_distance_index]
        found_nhood = 1
        if found_nhood == 0:
            sys.exit("I'm sorry I can't do that Dave...")
    return nhood
    
start_time = time.time()

from tqdm.auto import tqdm, trange
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as pool:
    data["NEIGHBOURHOOD"] = list(tqdm(pool.map(find_county, data['X'], data['Y'], chunksize=10), total=data_df_01.shape[0]))
    
        
elapsed_time = time.time() - start_time
print("Multi-thread elapsed time: "+str(strftime("%H:%M:%S", gmtime(elapsed_time))))

In [61]:
#Importing Dataset
data_df = pd.read_csv('data//data-collisions.csv')

In [53]:
# choosing the data we will work with
col_set = ["SEVERITYCODE", "SPEEDING","ROADCOND"]
data_df = data_df[col_set]

In [23]:
# obtaining data dimensions
for feature in ["SPEEDING", "ROADCOND"]:
    print(data_df[feature].unique())

[nan 'Y']
['Wet' 'Dry' nan 'Unknown' 'Snow/Slush' 'Ice' 'Other' 'Sand/Mud/Dirt'
 'Standing Water' 'Oil']


In [25]:
data_df['SPEEDING'] = data_df['SPEEDING'].fillna('N')


#we replace the value Nan declaring it as unknown too
data_df['ROADCOND'] = data_df['ROADCOND'].fillna('Unknown')

# checking value once again...
for feature in ["SPEEDING", "ROADCOND"]:
    print(data_df[feature].unique())

['N' 'Y']
['Wet' 'Dry' 'Unknown' 'Snow/Slush' 'Ice' 'Other' 'Sand/Mud/Dirt'
 'Standing Water' 'Oil']


In [26]:
# We assign new values to roadcond
data_df['ROADCOND'].replace(to_replace=['Wet','Dry','Unknown','Snow/Slush','Ice','Other','Sand/Mud/Dirt','Standing Water','Oil'], value = ['Dangerous','Normal','Normal','Dangerous','Dangerous','Normal','Dangerous','Dangerous','Dangerous'], inplace=True)


In [27]:
data_df.head()

,SEVERITYCODE,SPEEDING,ROADCOND
0,2,N,Dangerous
1,1,N,Dangerous
2,1,N,Normal
3,1,N,Normal
4,2,N,Dangerous


In [28]:
data_df.shape

(194673, 3)